# Gradient Descent Variants

- SGD
- Momentum
- Nestrov 
- Adagrad
- Adadelta
- Adam

In [1]:
# !pip install torch torchvision

## 1. Settings
### 1) Import required libraries

In [2]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### 2) Set hyperparameters

In [3]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

## 2. Data

### 1) Download Data

In [4]:
mnist_train = dset.MNIST("./", 
                         train = True,
                          download = True, 
                         transform = transforms.ToTensor(), 
                         target_transform = None)
mnist_test = dset.MNIST("./", 
                        train = False,
                        download = True, 
                        transform = transforms.ToTensor(), 
                        target_transform = None)

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw


Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


### 2) Check Dataset

In [5]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
print(mnist_test.__getitem__(0)[0].size(), mnist_test.__len__())

torch.Size([1, 28, 28]) 60000
torch.Size([1, 28, 28]) 10000


### 3) Set DataLoader

In [6]:
train_loader = torch.utils.data.DataLoader(mnist_train,
                                           batch_size = batch_size, 
                                           shuffle = True,
                                           num_workers = 2,
                                           drop_last = True)
test_loader = torch.utils.data.DataLoader(mnist_test,
                                          batch_size = batch_size,
                                          shuffle = False,
                                          num_workers = 2,
                                          drop_last = True)

## 3. Model & Optimizer

### 1) CNN Model

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding = 1),    # 28 x 28
            nn.ReLU(),
            nn.Conv2d(16, 32, 3, padding = 1),   # 28 x 28
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                  # 14 x 14
            nn.Conv2d(32, 64, 3, padding = 1),   # 14 x 14
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                   # 7 x 7
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64 * 7 * 7, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
        )        
        
    def forward(self, x):
        out = self.layer(x)
        out = out.view(batch_size, -1)
        out = self.fc_layer(out)
        return out

### 2) Loss func & Optimizer

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
# 다양한 경사하강법 알고리즘이 파이토치에 이미 구현이 되어 있기 때문에, 단순하게 이름만 바꿈으로써 해당 알고리즘을 쓸 수 있습니다.
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

cuda:0


## 4. Train 

In [9]:
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x = image.to(device)
        y_ = label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()
        
    if i % 10 == 0:
        print(loss)                          

tensor(0.3125, device='cuda:0', grad_fn=<NllLossBackward>)


In [10]:
param_list = list(model.parameters())
print(param_list)

[Parameter containing:
tensor([[[[ 0.2398, -0.0054,  0.3581],
          [ 0.3284, -0.0672,  0.2573],
          [-0.1768,  0.1994, -0.0231]]],


        [[[ 0.1781, -0.2279,  0.2782],
          [ 0.0561,  0.4064,  0.2319],
          [ 0.2315, -0.1537,  0.0422]]],


        [[[ 0.0562,  0.0646, -0.2304],
          [ 0.2670, -0.0527,  0.0951],
          [ 0.3216, -0.0499, -0.2944]]],


        [[[-0.0319,  0.1560, -0.0436],
          [ 0.3844,  0.4032, -0.0811],
          [-0.2437,  0.1156,  0.1704]]],


        [[[ 0.2161,  0.2981, -0.0900],
          [ 0.1575, -0.1191,  0.2535],
          [ 0.2718, -0.0180,  0.3798]]],


        [[[-0.1461, -0.0485, -0.0211],
          [ 0.0447,  0.1511, -0.2355],
          [ 0.2807,  0.0345,  0.1556]]],


        [[[ 0.1055,  0.3657,  0.2350],
          [ 0.1480, -0.1737, -0.1697],
          [-0.3457, -0.0863,  0.2849]]],


        [[[ 0.3730,  0.2060,  0.3233],
          [-0.2287, -0.0279, -0.0583],
          [ 0.0081, -0.1240, -0.0256]]],


        [

## 5. Test

In [11]:
correct = 0
total = 0

with torch.no_grad():
  for image, label in test_loader:
      x = image.to(device)
      y_ = label.to(device)

      output = model.forward(x)
      _, output_index = torch.max(output, 1)

      total += label.size(0)
      correct += (output_index == y_).sum().float()

  print("Accuracy of Test Data: {}".format(100 * correct / total))

Accuracy of Test Data: 98.52764892578125
